# Chapter 15 - Natural Language Processing: Pretraining

## 15.1. Word Embedding (word2vec)

In natural language processing, words are the basic unit of the meaning. The *word vectors* are vectors used to represent words, and can also be considered as feature vectors or representations of words.

The technique of mapping words to real vectors is called *word embedding*. 

### 15.1.1. One-Hot Vectors Are a Bad Choice

Suppose that the number of different words in the dictionary (the dictionary size) is $N$, and each word corresponds to a different integer (index) from 0 to $N-1$. To obtain the one-hot vector representation for any word with index $i$, we create a length-$N$ vector with all 0s and set the element at position $i$ to 1.

However, one-hot word vectors cannot accurately express the similarity between different words, such as the *cosine similarity*. For vectors $\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$, their cosine similarity is the cosine of the angle between them:
\begin{split}
\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1].
\end{split}
Since the cosine similarity between one-hot vectors of any two different words is 0, one-hot vectors cannot encode similarities among words.

### 15.1.2. Self-Supervised word2vec

The word2vec algorithm maps each word to a fixed-length vector, and thes vectors can better express the similarity and analogy relationships among different words.

The word2vec algorithm contains two models:
* *skip-gram*, and
* *continuous bag of words (CBOW)*.

Since supervision comes from the data without labels, both skip-gram and continuous bag of words are self-supervised learning models.

### 15.1.3. The Skip-Gram Model

The *skip-gram* model assumes that *a word can be used to generate its surrounding words in a text squence*.

For example, suppose a text squence, "the", "man", "loves", "his", "son". If "loves" is chosen as the *center word* and we set the context window size to 2. As shown in the figure below, given the center word "loves", the skip-gram model considers the conditional probability for generating the *context words*: "the", "man", "his", "son", which are no more than 2 words away from the center word:
\begin{split}
P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}).
\end{split}

![](../imgs/ch15/skip-gram.svg)

Assume that the context words are independently generated given the center words (i.e., conditional independence). Then, the above conditional probability can be rewritten as
\begin{split}
P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}) = P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}).
\end{split}

Each word in this model has two $d$-dimensional vector representations to calculate conditional probabilities.

For any word with index $i$ in the dictionary, denote by $\mathbf{v}_i \in \mathbb{R}^d$ and $\mathbf{u}_i \in \mathbb{R}^d$ its two vectors when used as a *center word* and a *context word*, respectively. The conditional probability of generating any context word $w_o$ (with index $o$ in the dictionary) given the center word $w_c$ (with index $c$ in the dictionary) can be modeled by a softmax operation on vector dot products:
\begin{split}
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
\end{split}
where the vocabulary index set is $\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$.

Given a text sequence of length $T$, where the word at time step $t$ is denoted by $w^{(t)}$. Assume that context words are independently generated given any center word. For context window size $m$, the likelihood function of the skip-gram model is the probability of generating all context words given any center word:
\begin{split}
\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),
\end{split}
where any time step that is less than 1 or greater than $T$ can be ignored.

#### 15.1.3.1. Training

The model parameters in the skip-gram model are the center word vectors and the context word vectors for each word in the vocabulary.

During training, we learn the model parameters by maximizing the likelihood function (i.e., maximum likelihood estimation), which is equivalent to minimizing the following loss function:
\begin{split}
- \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \textrm{log}\, P(w^{(t+j)} \mid w^{(t)}).
\end{split}

Using SGD to minimize this loss function, in each iteration, we can randomly sample a shorter subsequence to calculate the (stochastic) gradient for this subsequence to update the model parameters.

The (stochastic) gradients are the gradients of the log conditional probability with respect to the center word vector and the context word vector. Recall in the previous section that the log conditional probability involving any pair of the center word $w_c$ and the context word $w_o$ is
\begin{split}
\log P(w_o \mid w_c) =\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right).
\end{split}

Its gradient with respect to the center word vector $\mathbf{v}_c$ is
\begin{split}
\begin{aligned}
\frac{\partial \textrm{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c}&= \mathbf{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\exp(\mathbf{u}_j^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\right) \mathbf{u}_j\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
\end{aligned}
\end{split}
which requries the conditional probabilities of all words in the dictionary with $w_c$ as the center word.

After training, for any word with index $i$ in the dictionary, we obtain both word vectors $\mathbf{v}_i$ (as the center word) and $\mathbf{u}_i$ (as the context word).

In NLP, the center word vectors of the skip-gram model are typically used as the word representations.

### 15.1.4. The Continuous Bag of Words (CBOW) Model

The *continuous bag of words (CBOW)* model is similar to the skip-gram model. The main difference is that the CBOW model assumes that *a center word is generated based on its surrounding context words in the text sequence*.

Using the same text sequence, "the", "man", "loves", "his", "son", with "loves" as the center word and the context window size being 2, the continuous bag of words model consideres the conditional probability of generating the center word "loves" based on the context words "the", "man", "his", "son":
\begin{split}
P(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}).
\end{split}
as shown in the figure below.

![](../imgs/ch15/cbow.svg)

Since there are multiple context words in the CBOW model, these context word vectors are averaged in the calculation of the conditional probability.

For any word with index $i$ in the dictionary, denote by $\mathbf{v}_i \in \mathbb{R}^d$ and $\mathbf{u}_i \in \mathbb{R}^d$ its two vectors when used as a *context word* and a *center word*, respectively, (the reverse of the skip-gram model). The conditional probability of generating any center word $w_c$ (with index $c$ in the dictionary) given the context words $w_{o_1}, \ldots, w_{o_{2m}}$ (with indices $o_1, \ldots, o_{2m}$ in the dictionary) can be modeled by
\begin{split}
P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\exp\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \exp\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}) \right)}.
\end{split}

For brevity, let $\mathcal{W}_o = \{w_{o_1}, \ldots, w_{o_{2m}}\}$ denote the set of context words and $\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots + \mathbf{v}_{o_{2m}}\right)/(2m)$ denote their average. Then, the above conditional probability can be rewritten as
\begin{split}
P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}.
\end{split}

Given a text sequence of length $T$, where the word at time step $t$ is denoted by $w^{(t)}$. For context window size $m$, the likelihood function of the CBOW model is the probability of generating all center words given any context words:
\begin{split}
\prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).
\end{split}

#### 15.1.4.1. Training

Training the CBOW model is similar to training the skip-gram model.

The maximum likelihood estimation of the CBOW model is equivalent to minimizing the following loss function:
\begin{split}
-\sum_{t=1}^T  \textrm{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).
\end{split}

Note that
\begin{split}
\log\,P(w_c \mid \mathcal{W}_o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right).
\end{split}

Hence, its gradient with respect to the context word vector $\mathbf{v}_{o_i}$ ($i = 1, \ldots, 2m$) is
\begin{aligned}
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} &= \frac{1}{2m} \left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)} \right) \\
&= \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right).
\end{aligned}

Unlike the skip-gram model, the continuous bag of words model typically uses context word vectors as the word representations.

## 15.2. Approximate Training

The gradient calculation of the skip-gram and CBOW models involves the sum of all conditional probabilities of words in the dictionary. When the dictionary is large, the sum of all conditional probabilities is computationally expensive.

To reduce the aforementioned computational cost, we can use *negative sampling* and *hierarchical softmax* to approximate the maximum likelihood estimation of the skip-gram and CBOW models.

### 15.2.1. Negative Sampling

*Negative sampling* modifies the original objective function.

Given the context window of a center word $w_c$, the fact that any (context) word $w_o$ comes from this context window is considered as an event with the probability modeled by
\begin{split}
P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c),
\end{split}
where $\sigma$ is the sigmoid function:
\begin{split}
\sigma(x) = \frac{1}{1+\exp(-x)}.
\end{split}

Given a text sequence of lenght $T$, denote by $w^{(t)}$ the word at time step $t$ and let the context window size be $m$, the joint probability of generating all context words in the text sequence is
\begin{split}
\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)}).
\end{split}

This joint probability is maximized to 1 only if all the word vectors are equal to infinity. To make the objective function more meaningful, *negative sampling* adds negative examples sampled from a predefined distribution.

Denote by $S$ the event that a context word $w_o$ comes from the context window of a center word $w_c$. 

For this event involving $w_o$, from a predefined distribution $P(w)$, we can sample $K$ *noise words* that are not from this context window. Denote by $N_k$ the event that a noise word $w_k$ ($k = 1, \ldots, K$) does not come from the context window of $w_c$.

Assume that these events involving both the positive example $S$ and the negative examples $N_1, \ldots, N_K$ are mutually independent. Then, the negative sampling rewrites the joint probability (involving only positive examples $S$) as
\begin{split}
\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)}) = \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),
\end{split}
where the conditional probability is approximated through events $S, N_1, \ldots, N_K$:
\begin{split}
P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k).
\end{split}

Denote by $i_t$ and $h_k$ the indices of a word $w^{(t)}$ at time step $t$ of a text sequence and a noise word $w_k$, respectively. The logarithmic loss with respect to the conditional probabilities of the positive example and the negative examples is
\begin{split}\begin{aligned}
-\log P(w^{(t+j)} \mid w^{(t)})
=& -\log P(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)\right)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right).
\end{aligned}\end{split}

The computational cost for gradients at each training step does not depend on the dictionary size, but linearly depends on $K$. When setting the hyperparameter $K$ to a smaller value, the computational cost for gradients at each training with negative sampling is smaller.

### 15.2.2. Hierarchical Softmax

The *hierarchical softmax* uses a binary where each leaf node of the tree represents a word in the dictionary $\mathcal{V}$, as shown in the figure below.

![](../imgs/ch15/hi-softmax.svg)

Denote by $L(w)$ the number of nodes (including both ends) on the path from the root node to the leaf node representing word $w$ in the binary tree.

Let $n(w,j)$ be the $j$-th node on this path, with its context word vector being $\mathbf{u}_{n(w,j)}$. For instance, in the figure above, $L(w_3) = 4$ since the path from the root node to the leaf node representing $w_3$ is $n(w_3, 1)$, $n(w_3, 2)$, $n(w_3, 3)$ and $w_3$.

The *hierarchical softmax* approximates the conditional probability 
\begin{split}
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
\end{split}
as
\begin{split}
P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma\left( [\![  n(w_o, j+1) = \textrm{leftChild}(n(w_o, j)) ]\!] \cdot \mathbf{u}_{n(w_o, j)}^\top \mathbf{v}_c\right),
\end{split}
where $\sigma$ is the sigmoid function and $\textrm{leftChild}(n)$ is the left child node of $n$: if $x$ is true, then $[\![x]\!] = 1$; otherwise, $[\![x]\!] = -1$.

In the figure above, if we want to calculate the conditional probaility of generating word $w_3$ given word $w_c$, we need to calculate the dot products between the word vector $\mathbf{v}_c$ of $w_c$ and non-leaf node vectors on the path (the bold line segments in the figure above) from the root to $w_3$, which is traversed left, right, and then left:
\begin{split}
P(w_3 \mid w_c) = \sigma(\mathbf{u}_{n(w_3, 1)}^\top \mathbf{v}_c) \cdot \sigma(-\mathbf{u}_{n(w_3, 2)}^\top \mathbf{v}_c) \cdot \sigma(\mathbf{u}_{n(w_3, 3)}^\top \mathbf{v}_c).
\end{split}

Since $\sigma(x) + \sigma(-x) = 1$, it holds that the conditional probabilities of generating all the words in dictionary $\mathcal{V}$ based on any word $w_c$ sum up to one:
\begin{split}
\sum_{w \in \mathcal{V}} P(w \mid w_c) = 1.
\end{split}

Since $L(w_o) - 1$ is the order of $\mathcal{O}(\log_2 |\mathcal{V}|)$ due to the binary tree structure, when the dictionary size $|\mathcal{V}|$ is large, the computational cost for each training step of the hierarchical softmax is significantly reduced compared with that without approximate training.

## 15.3. The Dataset for Pretraining Word Embeddings